In [ ]:
import os
import sys
import ast
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow.compat.v1 as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from magenta.models.performance_rnn import performance_model
from magenta.models.performance_rnn import performance_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from note_seq import midi_io
from note_seq.protobuf import music_pb2, generator_pb2
import note_seq
import numpy as np

c:\Users\adamc\miniconda3\envs\magenta_legacy\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\Users\adamc\miniconda3\envs\magenta_legacy\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\Users\adamc\miniconda3\envs\magenta_legacy\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
 

✓ Libraries imported


In [6]:
# Configuration for Performance RNN generation
# NOTE 1 step = 10 ms

NUM_STEPS = 1000  # Number of steps to generate (10ms per step)
TEMPERATURE = 0.9

PRIMER_TYPE = 'melody'  # Options: 'melody', 'pitches', 'midi', or None

# Primer options (only used based on PRIMER_TYPE):
PRIMER_MELODY = "[60, 62, 64, 65, 67, 69, 71, 72]"  # C major scale (can use -2 for no event, -1 for note-off)
PRIMER_PITCHES = "[60, 64, 67]"  # C major chord starting pitches
PRIMER_MIDI = None  # Path to a MIDI file, e.g., "path/to/file.mid"

# Configuration model to use
CONFIG = 'performance'  # Options: 'performance', 'performance_with_dynamics', etc.

# For conditional models, you can set:
# NOTES_PER_SECOND = 4
# PITCH_CLASS_HISTOGRAM = "[2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]"  # C-major scale emphasis

print(f"🎵 Performance RNN Generation Configuration:")
print(f"   Config: {CONFIG}")
print(f"   Steps: {NUM_STEPS} (Duration: {NUM_STEPS * 0.01:.1f}s)")
print(f"   Temperature: {TEMPERATURE}")
print(f"   Primer Type: {PRIMER_TYPE}")
if PRIMER_TYPE == 'melody':
    print(f"   Primer Melody: {PRIMER_MELODY}")
elif PRIMER_TYPE == 'pitches':
    print(f"   Primer Pitches: {PRIMER_PITCHES}")
elif PRIMER_TYPE == 'midi':
    print(f"   Primer MIDI: {PRIMER_MIDI}")


🎵 Performance RNN Generation Configuration:
   Config: performance
   Steps: 1000 (Duration: 10.0s)
   Temperature: 0.9
   Primer Type: melody
   Primer Melody: [60, 62, 64, 65, 67, 69, 71, 72]


In [7]:
# Load the Performance RNN model and create generator
print("Loading Performance RNN model...")

# Disable TF v2 behavior
tf.compat.v1.disable_v2_behavior()

# Get configuration from available configs
# Available models: 'performance', 'performance_with_dynamics', 
# 'performance_with_dynamics_and_modulo_encoding',
# 'density_conditioned_performance_with_dynamics',
# 'pitch_conditioned_performance_with_dynamics',
# 'multiconditioned_performance_with_dynamics'
config = performance_model.default_configs[CONFIG]
config.hparams.parse('')  # Use default hyperparameters

# Download pre-trained bundle (models are auto-cached)
import urllib.request
import pathlib

model_cache_dir = pathlib.Path.home() / '.magenta' / 'models'
model_cache_dir.mkdir(parents=True, exist_ok=True)

bundle_file = model_cache_dir / f'{CONFIG}.mag'

if not bundle_file.exists():
    print(f"  Downloading pre-trained model ({CONFIG})...")
    bundle_url = f'http://download.magenta.tensorflow.org/models/{CONFIG}.mag'
    try:
        urllib.request.urlretrieve(bundle_url, str(bundle_file))
        print(f"  ✓ Downloaded to {bundle_file}")
    except Exception as e:
        print(f"  Error downloading: {e}")
        print(f"  Make sure CONFIG is set to a valid pre-trained model name")
        raise
else:
    print(f"  ✓ Using cached model: {bundle_file}")

# Load the bundle
bundle = sequence_generator_bundle.read_bundle_file(str(bundle_file))
print(f"  ✓ Bundle loaded")

# Create the model
model = performance_model.PerformanceRnnModel(config)

# Create the generator with bundle
generator = performance_sequence_generator.PerformanceRnnSequenceGenerator(
    model=model,
    details=config.details,
    steps_per_second=config.steps_per_second,
    num_velocity_bins=config.num_velocity_bins,
    control_signals=config.control_signals,
    optional_conditioning=config.optional_conditioning,
    checkpoint=None,
    bundle=bundle,
    note_performance=config.note_performance
)

print(f"✓ Model loaded: {CONFIG}")
print(f"  Steps per second: {config.steps_per_second}")
print(f"  Velocity bins: {config.num_velocity_bins if config.num_velocity_bins > 0 else 'Not quantized'}")


Loading Performance RNN model...
  ✓ Using cached model: C:\Users\adamc\.magenta\models\performance.mag
  ✓ Bundle loaded
✓ Model loaded: performance
  Steps per second: 100
  Velocity bins: Not quantized


In [8]:
# Prepare primer sequence based on PRIMER_TYPE
print("\n🎹 Preparing primer sequence...")

if PRIMER_TYPE is None:
    # Start from scratch with empty primer
    primer_sequence = music_pb2.NoteSequence()
    primer_sequence.ticks_per_quarter = note_seq.STANDARD_PPQ
    print("  Primer: Starting from scratch (random)")

elif PRIMER_TYPE == 'melody':
    # Create primer from melody event list
    # Format: -2 = no event, -1 = note-off, 0-127 = note-on at pitch
    primer_notes = ast.literal_eval(PRIMER_MELODY)
    primer_melody = note_seq.Melody(primer_notes)
    primer_sequence = primer_melody.to_sequence()
    print(f"  Primer: Melody with {len([p for p in primer_notes if p >= 0])} notes")

elif PRIMER_TYPE == 'pitches':
    # Create primer from chord pitches
    pitches = ast.literal_eval(PRIMER_PITCHES)
    primer_sequence = music_pb2.NoteSequence()
    primer_sequence.ticks_per_quarter = note_seq.STANDARD_PPQ
    
    # Add notes for each pitch as a chord
    for pitch in pitches:
        note = primer_sequence.notes.add()
        note.start_time = 0
        note.end_time = 60.0 / note_seq.DEFAULT_QUARTERS_PER_MINUTE  # Quarter note at 120 QPM
        note.pitch = pitch
        note.velocity = 100
    
    primer_sequence.total_time = primer_sequence.notes[0].end_time if primer_sequence.notes else 0
    print(f"  Primer: Chord with pitches {pitches}")

elif PRIMER_TYPE == 'midi':
    # Load primer from MIDI file
    if PRIMER_MIDI and os.path.exists(PRIMER_MIDI):
        primer_sequence = note_seq.midi_file_to_sequence_proto(PRIMER_MIDI)
        print(f"  Primer: Loaded from MIDI file ({len(primer_sequence.notes)} notes)")
    else:
        print(f"  Error: MIDI file not found. Using empty primer.")
        primer_sequence = music_pb2.NoteSequence()
        primer_sequence.ticks_per_quarter = note_seq.STANDARD_PPQ

# Create GeneratorOptions
generator_options = generator_pb2.GeneratorOptions()

# Calculate generate time
seconds_per_step = 1.0 / generator.steps_per_second
generate_end_time = NUM_STEPS * seconds_per_step

# Set generation section (from end of primer to desired length)
generate_section = generator_options.generate_sections.add()
generate_section.start_time = primer_sequence.total_time
generate_section.end_time = generate_end_time

# Set temperature
generator_options.args['temperature'].float_value = TEMPERATURE

print(f"\n🎼 Generating performance ({NUM_STEPS} steps = {generate_end_time:.2f}s)...")

# Generate the performance
generated_sequence = generator.generate(primer_sequence, generator_options)

print(f"✓ Generated {len(generated_sequence.notes)} notes")
print(f"✓ Duration: {generated_sequence.total_time:.2f} seconds")
print(f"✓ Events processed with proper note-on/off and velocity handling")



🎹 Preparing primer sequence...
  Primer: Melody with 8 notes

🎼 Generating performance (1000 steps = 10.00s)...
✓ Generated 86 notes
✓ Duration: 10.00 seconds
✓ Events processed with proper note-on/off and velocity handling


In [9]:
# Save the generated performance to MIDI file
output_file = f"performance_rnn_{CONFIG}.mid"

try:
    note_seq.sequence_proto_to_midi_file(generated_sequence, output_file)
    print(f"\n✅ Performance saved: {output_file}")
    print(f"\n📊 Generation Details:")
    print(f"   Config: {CONFIG}")
    print(f"   File: {output_file}")
    print(f"   Notes: {len(generated_sequence.notes)}")
    print(f"   Duration: {generated_sequence.total_time:.2f}s")
    print(f"   Temperature: {TEMPERATURE}")
    print(f"   Primer Type: {PRIMER_TYPE}")
    print(f"\n💡 Note: Performance RNN generates with:")
    print(f"   - Proper note-on/note-off events (not fixed duration)")
    print(f"   - Expressive timing (time-shift events in 10ms increments)")
    print(f"   - Velocity dynamics (for 'performance_with_dynamics' config)")
    print(f"\n🎵 Ready to listen! Open the MIDI file in any music player.")
except Exception as e:
    print(f"Error saving file: {e}")
    import traceback
    traceback.print_exc()

# Optional: For conditional models, you can set control signals:
# Example for pitch_conditioned model:
# generator_options.args['pitch_class_histogram'].string_value = "[2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]"
# (Pass these before calling generator.generate())



✅ Performance saved: performance_rnn_performance.mid

📊 Generation Details:
   Config: performance
   File: performance_rnn_performance.mid
   Notes: 86
   Duration: 10.00s
   Temperature: 0.9
   Primer Type: melody

💡 Note: Performance RNN generates with:
   - Proper note-on/note-off events (not fixed duration)
   - Expressive timing (time-shift events in 10ms increments)
   - Velocity dynamics (for 'performance_with_dynamics' config)

🎵 Ready to listen! Open the MIDI file in any music player.


## Advanced Features

### Performance RNN Model Types
- **performance**: Base model, ignores velocities, models note-on/off with expressive timing
- **performance_with_dynamics**: Includes velocity changes (quantized to 32 bins)
- **performance_with_dynamics_and_modulo_encoding**: Alternative encoding mapping values to unit circle

### Conditional Models (for controlling generation)
- **density_conditioned_performance_with_dynamics**: Control note density (notes_per_second)
- **pitch_conditioned_performance_with_dynamics**: Control pitch distribution (pitch_class_histogram)
- **multiconditioned_performance_with_dynamics**: Both density and pitch control

### Event Encoding
Performance RNN generates sequences of events:
- **NOTE_ON(pitch)**: Start a note (pitch 0-127)
- **NOTE_OFF(pitch)**: End a note
- **TIME_SHIFT(amount)**: Advance time by 10ms increments (up to 1 second)
- **VELOCITY(value)**: Set current velocity (0-127, quantized if using dynamics model)

### Key Parameters
- **num_steps**: Duration in 10ms units (3000 = 30 seconds)
- **temperature**: Randomness control (0.5 = conservative, 1.0 = normal, 1.5+ = very random)
- **primer_sequence**: Starting sequence (melody list, pitches, or MIDI file)

### Command-Line Equivalent
This notebook implements:
```bash
performance_rnn_generate \
  --config=performance_with_dynamics \
  --bundle_file=<auto-downloaded> \
  --output_dir=. \
  --num_outputs=1 \
  --num_steps=3000 \
  --primer_melody="[60,62,64,65,67,69,71,72]"
```


In [ ]:
# Example: Using Conditional Models
# Uncomment and modify to use conditional models for controlled generation

# For density_conditioned_performance_with_dynamics:
# NOTES_PER_SECOND = 4
# conditional_params = {
#     'notes_per_second': NOTES_PER_SECOND
# }

# For pitch_conditioned_performance_with_dynamics:
# PITCH_CLASS_HISTOGRAM = "[2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]"  # Emphasize C major
# conditional_params = {
#     'pitch_class_histogram': PITCH_CLASS_HISTOGRAM
# }

# For multiconditioned_performance_with_dynamics:
# conditional_params = {
#     'notes_per_second': 4,
#     'pitch_class_histogram': "[2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]"
# }

# Note: These parameters guide but don't strictly enforce the model's output
# They work globally, affecting the entire generated performance

print("💡 To use conditional models:")
print("   1. Change CONFIG to a conditional model name")
print("   2. Define notes_per_second and/or pitch_class_histogram")
print("   3. Pass them to the generate() function as control parameters")
